In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
data = pd.read_csv('we_fn_use_c_marketing_customer_value_analysis.csv')
display(data.shape)
data.head()

In [ ]:
data.columns = data.columns.str.lower()
data.columns = data.columns.str.replace(" ", "_")
data.head()

In [ ]:
data['effective_to_date'] = pd.to_datetime(data.effective_to_date,format="%Y-%M-%D")
data.head()

In [ ]:
data['effective_to_date'].dtype

In [ ]:
numerical_df = data.select_dtypes(np.number)
numerical_df.head()

In [ ]:
numerical_df.nunique()

In [ ]:
#dataframe understanding: 
# discrete columns : monthly_premium_auto, months_since_last_claim, months_since_policy_inception, number_of_open_complaints, number_of_policies
# continuous columns : income, customer_lifetime_value, total_claim_amount 

In [ ]:
numerical_df.monthly_premium_auto.value_counts()

In [ ]:
numerical_df.nunique()/len(numerical_df)

In [ ]:
def separate_numerical_columns(df):
    # Calculate the ratio of unique values to the total number of rows
    unique_ratio = df.nunique() / len(df)
    # Filter columns based on the condition
    continuous_columns = unique_ratio[unique_ratio > 0.05].index
    discrete_columns = unique_ratio[unique_ratio <= 0.05].index
    # Create new dataframes for discrete and continuous columns
    continuous_df = df[continuous_columns]
    discrete_df = df[discrete_columns]
    
    return continuous_df, discrete_df

continuous_df, discrete_df = separate_numerical_columns(numerical_df)

In [ ]:
print(f"Continuous columns shape: {continuous_df.shape}")
print(f"Discrete columns shape: {discrete_df.shape}")

In [ ]:
continuous_df.head()

In [ ]:
discrete_df.head()

In [ ]:
correlations_matrix = numerical_df.corr()
sns.heatmap(correlations_matrix, annot=True)
plt.show()

In [ ]:
# total_claim_amount is impacted by custemor lifetime value and the monthly premium auto. 
# additionally monthly premium auto is correlated to custemor lifetime value, which will probably skew the model 

In [ ]:
def plots(df):
    for column in continuous_df.columns:
        sns.boxplot(x=column, data=continuous_df)
        plt.title(f'Distribution of {column} (continuous - box plot)')
        plt.show()
    for column in discrete_df.columns:
        sns.displot(x=column, data=discrete_df, bins=50, kde=True)
        plt.title(f'Distribution of {column} (discrete - displot)')
        plt.show()

In [ ]:
plots(numerical_df)

In [ ]:
# There are outliers in the continuous columns - total claim amount and custemor lifetime value. 
# There are outliers in the descrete columns - monthly premium auto and number of open complaints.  

In [ ]:
display(data['total_claim_amount'].value_counts(dropna=False))

In [ ]:
continuous_df.describe()

In [ ]:
continuous_df.isna().sum()

In [ ]:
outliers_df = numerical_df[['monthly_premium_auto','income', 'total_claim_amount','customer_lifetime_value']]
outliers_df.describe().T

In [ ]:
clt_labels = ['Very Low','Low', 'Moderate', 'High', 'Very High']
numerical_df['clt_categories'] = pd.cut(numerical_df['customer_lifetime_value'],5,labels=clt_labels)

In [ ]:
numerical_df.clt_categories.value_counts()

In [ ]:
numerical_df.loc[numerical_df['clt_categories'] == numerical_df['clt_categories'].max()]

In [ ]:
# the customer_lifetime_value outliers are very few and versitile , we decided to keep them

In [ ]:
numerical_df['income'].hist(bins=10)

In [ ]:
# the income STD (and plot show) is aldo vastly different and the big amount of zero's are worth filling with the mean

In [ ]:
numericals_with_income = numerical_df[numerical_df['income']>0]
numericals_with_income.shape

In [ ]:
numericals_with_income.head()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler

In [ ]:
X_income = numericals_with_income.drop(['income','total_claim_amount','clt_categories'],axis=1) 
y_income = numericals_with_income['income']

In [ ]:
X_normalized_income = pd.DataFrame(MinMaxScaler().fit_transform(X_income), columns=X_income.columns)
X_normalized_income

In [ ]:
display(X_normalized_income.shape)
display(y_income.shape)

In [ ]:
X_all = numericals_with_income.drop(['income','total_claim_amount','clt_categories'],axis=1)
X_normalized_all = pd.DataFrame(MinMaxScaler().fit_transform(X_all), columns=X_all.columns)
y_all = numerical_df['income']

In [ ]:
KNN = KNeighborsClassifier(n_neighbors=3)
KNN.fit(X_normalized_income,y_income)
KNN.score(X_normalized_income,y_income)

In [ ]:
predicted_income = pd.Series(KNN.predict(X_normalized_all),name='predicted_income')

In [ ]:
predicted_income.value_counts(dropna=False)

In [ ]:
numericals_with_income['income'].value_counts(dropna=False)

In [ ]:
# the prediction is not very good, therfore we keep the predicted column for future model creation
# and fill in the prediction values where the value is 0

In [ ]:
numerical_df = pd.concat([numerical_df,predicted_income],axis=1)

In [ ]:
numerical_df.shape

In [ ]:
numerical_df['final_income'] =  np.where(numerical_df['income']==0, numerical_df['predicted_income'], numerical_df['income'])
numerical_df[['income','predicted_income','final_income']].head()

In [ ]:
# numerical_df.to_csv('C:\\Users\\Dana\\IH_Labs\\lab-cleaning-numerical-data\\numerical_df.csv', index=False)

## Lab | Cleaning categorical data

In [ ]:
data.head()

In [ ]:
categorical_df = data.select_dtypes('object')
categorical_df.head()

In [ ]:
categorical_df.head()

In [ ]:
categorical_df.nunique()

In [ ]:
categorical_df.isna().sum()/len(categorical_df)

In [ ]:
display(categorical_df['state'].value_counts(dropna=False))

In [ ]:
display(categorical_df['response'].value_counts(dropna=False))

In [ ]:
display(categorical_df['coverage'].value_counts(dropna=False))

In [ ]:
display(categorical_df['education'].value_counts(dropna=False))

In [ ]:
display(categorical_df['employmentstatus'].value_counts(dropna=False))

In [ ]:
display(categorical_df['gender'].value_counts(dropna=False))

In [ ]:
display(categorical_df['location_code'].value_counts(dropna=False))

In [ ]:
display(categorical_df['marital_status'].value_counts(dropna=False))

In [ ]:
display(categorical_df['policy_type'].value_counts(dropna=False))

In [ ]:
display(categorical_df['policy'].value_counts(dropna=False))

In [ ]:
display(categorical_df['renew_offer_type'].value_counts(dropna=False))

In [ ]:
display(categorical_df['sales_channel'].value_counts(dropna=False))

In [ ]:
display(categorical_df['vehicle_class'].value_counts(dropna=False))

In [ ]:
display(categorical_df['vehicle_size'].value_counts(dropna=False))

In [ ]:
categorical_df.dtypes
#all column are of object type as expected for a dataframe where numical columns were extruded

In [ ]:
categorical_df['vehicle_class'] = categorical_df['vehicle_class'].replace({'Sports Car':'Luxury', 'Luxury SUV':'Luxury', 'Luxury Car':'Luxury'})
#vehicle_class column has a small number of 'luxury cars' which can be combined in the interest of having less options and more clarity
display(categorical_df['vehicle_class'].value_counts(dropna=False))

In [ ]:
# when compare policy_type and policy, it is clear that the policy column sums up to the policy types column accordingly.
# therfor, it might not be neccesary to use the 'policy'table(i.e. drop the dolumn) as the information is too detailed. 

In [ ]:
display(categorical_df[['policy_type','policy']].value_counts(dropna=False))

In [ ]:
categorical_df == categorical_df.drop('policy', axis=1, inplace=True)
categorical_df.head()

In [ ]:
# categorical_df.to_csv('C:\\Users\\Dana\\IH_Labs\\lab-cleaning-numerical-data\\categorical_df.csv', index=False)

## Lab | Feature extraction

In [ ]:
categorical_df = categorical_df.drop(['customer'],axis=1) 
categorical_df

In [ ]:
def cat_plots(df):
    for column in categorical_df.columns:
        sns.countplot(x=column, data=categorical_df)
        plt.show()
cat_plots(categorical_df)

In [ ]:
# Columns to drop :
# Policy is dropped (also in previos lab), the policy type holds the totals, so it is redundant detailing. 
# Customer is not neccesary to predict the target.

In [ ]:
con_data= pd.concat([categorical_df,numerical_df],axis=1)
con_data['effective_to_date']=data['effective_to_date']
data['month']=data['effective_to_date'].dt.month
con_data['month']=data['month']
data['weekday']=data['effective_to_date'].dt.weekday
con_data['weekday']=data['weekday']
con_data

In [ ]:
sns.countplot(x='month', data=con_data)
plt.show()

In [ ]:
sns.countplot(x='weekday', data=con_data)
plt.show()

In [ ]:
# Comparitvly January is more active than February. 
# most contracts started on monday, second higher active day is Saturday. 


#con_data.to_csv('C:\\Users\\Dana\\IH_Labs\\lab-cleaning-numerical-data\\con_data.csv', index=False)

## Lab Comparing Regression Models


In [ ]:
y = con_data['target (total_claim_amount)']
X = con_data['features']
